### CIFAR-10 Image Classification using the Pretrained GoogLeNet in Pytorch

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the GPU
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}") #prints the name of the zero index gpu.
else:
    device = torch.device("cpu")  # Use the CPU
    print("No GPU available, using CPU instead.")

There are 1 GPU(s) available.
GPU device name: NVIDIA GeForce RTX 3060


In [3]:
# 1. Load CIFAR-10 and Define Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match GoogleNet input
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # ImageNet normalization
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# 2. Load Pretrained GoogleNet
googlenet = models.googlenet(pretrained=True)

googlenet.to(device) # Moves the model to the gpu.

# 3. Modify the Fully Connected Layer for CIFAR-10
num_ftrs = googlenet.fc.in_features
#googlenet.fc = nn.Linear(num_ftrs, 10)  # 10 output classes for CIFAR-10
googlenet.fc = nn.Linear(num_ftrs, 10).to(device) #move the new layer to the device as well.


# 4. Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(googlenet.parameters(), lr=0.001, momentum=0.9)

In [8]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_params(googlenet)

5610154

In [9]:
# 5. Training Loop
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # Move data to GPU
        optimizer.zero_grad()
        outputs = googlenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.096
[1,  4000] loss: 0.734
[1,  6000] loss: 0.633
[1,  8000] loss: 0.545
[1, 10000] loss: 0.505
[1, 12000] loss: 0.490
[2,  2000] loss: 0.385
[2,  4000] loss: 0.377
[2,  6000] loss: 0.372
[2,  8000] loss: 0.358
[2, 10000] loss: 0.339
[2, 12000] loss: 0.346
[3,  2000] loss: 0.265
[3,  4000] loss: 0.263
[3,  6000] loss: 0.256
[3,  8000] loss: 0.259
[3, 10000] loss: 0.261
[3, 12000] loss: 0.271
[4,  2000] loss: 0.198
[4,  4000] loss: 0.199
[4,  6000] loss: 0.192
[4,  8000] loss: 0.208
[4, 10000] loss: 0.195
[4, 12000] loss: 0.210
[5,  2000] loss: 0.145
[5,  4000] loss: 0.171
[5,  6000] loss: 0.156
[5,  8000] loss: 0.161
[5, 10000] loss: 0.163
[5, 12000] loss: 0.167
Finished Training


In [10]:
# 6. Evaluation
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device) # Move data to GPU
        outputs = googlenet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 91 %


In [11]:
# 6. Evaluation
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device) # Move data to GPU
        outputs = googlenet(images)
        _, predictions = torch.max(outputs, 1)
        c = (predictions == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_name = classes[label]
            total_pred[class_name] += 1
            correct_pred[class_name] += c[i].item()

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device) # Move data to GPU
        outputs = googlenet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Overall Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Accuracy for class: plane is 91.6 %
Accuracy for class: car   is 97.7 %
Accuracy for class: bird  is 88.4 %
Accuracy for class: cat   is 85.5 %
Accuracy for class: deer  is 95.7 %
Accuracy for class: dog   is 87.8 %
Accuracy for class: frog  is 90.8 %
Accuracy for class: horse is 91.8 %
Accuracy for class: ship  is 91.6 %
Accuracy for class: truck is 92.5 %
Overall Accuracy of the network on the 10000 test images: 91.39 %
